进入决赛后夺冠以来分析

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
summary = spark.read.format("csv").option("header",True).option("encoding","utf-8").option("sep",",").load("../dataset/WorldCupsSummary.csv")
summary.createTempView("summary_tb")
summary.head()

Row(Year='1930', HostCountry='Uruguay', Winner='Uruguay', Second='Argentina', Third='USA', Fourth='Yugoslavia', GoalsScored='70', QualifiedTeams='13', MatchesPlayed='18', Attendance='590549', HostContinent='America', WinnerContinent='America')

In [9]:
percentage_of_winning_final = spark.sql('''
SELECT IFNULL(Winner,Second) AS country, ROUND(IFNULL((win_num/(win_num+sec_num))*100,0),2) AS percentage FROM
(SELECT Winner,COUNT(Winner) AS win_num FROM summary_tb GROUP BY Winner) AS t1
FULL OUTER JOIN 
(SELECT Second,COUNT(Second) AS sec_num FROM summary_tb GROUP BY Second) AS t2
ON t1.Winner=t2.Second ORDER BY percentage DESC
''').show()

+--------------+----------+
|       country|percentage|
+--------------+----------+
|        Brazil|     71.43|
|        France|     66.67|
|         Italy|     66.67|
|       Germany|      50.0|
|    Germany FR|      50.0|
|     Argentina|      40.0|
|       Croatia|       0.0|
|Czechoslovakia|       0.0|
|       England|       0.0|
|       Hungary|       0.0|
|   Netherlands|       0.0|
|         Spain|       0.0|
|        Sweden|       0.0|
|       Uruguay|       0.0|
+--------------+----------+
